# Simple script to input, calculate, and output

Note: in Jupyter notebooks, we can make comments in Markdown cells.  So making inline comments (lines that start with the hash character # and are ignored by Python) is less important.  

Here we set up the data needed to run the calculation

In [ ]:
# This is an inline comment that will be ignored by Python
pi = 3.14159
print(pi)
diameter = float(input('Enter the diameter, then press the Enter/return key: '))  # when this line runs you need to enter a number
print(diameter)

The data need to be transformed into a usable form

In [ ]:
radius = diameter/2
print(radius)

We have all of the information necessary to complete the calculation

In [ ]:
print('The area of the circle is ',pi*radius**2)